# Stacjonarność


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import statsmodels.api as sm

Wczytajmy dane

In [ ]:
import pandas as pd
data = pd.read_csv('data/AirPassengers.csv', index_col = 'Month', squeeze = True, parse_dates=['Month'])
print(data.head())

In [ ]:
data['1949-01-01':'1949-05-01']

In [ ]:
data.index

# Jak sprawdzić stacjonarność szergu czasowego?
 
Stacjonarność jest określona przy użyciu bardzo skomplikowanych kryteriów. Jednak w celach praktycznych można założyć, że szereg czasowy jest stacjonarny, jeśli ma stałe właściwości statystyczne w czasie, tj. następujące:

- stała średnia
- stała wariancja
- autokowariancja nie zależy od czasu.


In [ ]:
plt.figure(figsize=(12,6))
data.loc['1949':].plot()
plt.show()

Jest oczywiste, że istnieje ogólna tendencja wzrostowa w danych wraz z sezonowymi wahaniami. Jednak nie zawsze możliwe jest takie wizualne wnioskowanie (zobaczymy takie przypadki później). Więc bardziej formalnie możemy sprawdzić stacjonarność używając:

- możemy wyznaczyć średnią ruchomej lub ruchomą wariancji i sprawdzić, czy zmienia się ona z czasem. Średnia ruchoma lub ruchoma wariancja oznacza, że w każdej chwili 't' przeanalizujemy średnią /wariancję z ostatniego roku, tzn. w ciągu ostatnich 12 miesięcy. Ale znowu jest to bardziej wizualna technika.

- Test Dickey-Fuller: jest to jeden ze statystycznych testów sprawdzających stacjonarność. Tutaj hipoteza zerowa to, że szereg czasowy jest niestacjonarny. 
Wyniki testu obejmują statystykę testu oraz niektóre wartości krytyczne dla różnych poziomów ufności. Jeśli statystyka testowa jest mniejsza niż  wartość krytyczna, możemy odrzucić hipotezę zerową i powiedzieć, że szereg czasowy jest stacjonarny. 

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Zadanie 1
Uzupełnij poniższą funkcję, tak żeby dla podanych danych tworzony był wykres szeregu czasowego, średniej ruchomej i ruchomej wariancji oraz wyświetlane były informacje z testu Dickey'a-Fullera

In [ ]:
def test_stationarity(timeseries):
    rolmean = timeseries.rolling(12).mean()#.rolling_mean(timeseries, window=12)
    rolstd = timeseries.rolling(12).std()
    


In [ ]:
test_stationarity(data)

# Jak przetrasformować szerg czasowy tak by był stacjonarny?

Chociaż założenie stacjonarności jest przyjmowane w wielu modelach TS, w praktyce żaden szereg czasowy nie jest stacjonarny. Mamy sposoby, aby szereg stał się stacjonarny. Należy pamiętać, że jest to prawie nie możliwe, ale staramy się doprowadzić do sytuacji w której jest on jak najbliżej bycia stacjonarnym.

Istnieją dwa główne powody, które powodują niestacjonarność:

1. Trend - zmienna średnia w czasie. Na przykład, w naszym przypadku zauważyliśmy, że przeciętnie liczba pasażerów rośnie z upływem czasu.

2. Sezonowość - zmiany w określonych przedziałach czasowych. Np. Ludzie mogą mieć skłonność do zakupu samochodów w danym.

Podstawową zasadą jest modelowanie lub szacowanie trendu i sezonowości w szeregu czasowym oraz usunięcie ich, aby uzyskać stacjonarne szeregi czasowe. Następnie dla takiego szeregu czasowego można zastosować modele statystyczne. Ostatnim krokiem byłoby przekształcenie prognozowanych wartości do pierwotnej skali poprzez dodanie trendu i sezonowości.

# Czasami pomaga wzięcie logarytmu z danych.

In [ ]:
ts_log = np.log(data)
plt.figure(figsize=(12,6))
plt.plot(ts_log)
plt.show()

# Odejmijmy średnią kroczącą.

In [ ]:
moving_avg = ts_log.rolling(12).mean()
plt.figure(figsize=(12,6))
plt.plot(ts_log - moving_avg)
moving_avg1 = (ts_log - moving_avg).rolling(12).mean()
plt.plot(moving_avg1, color='red')
plt.show()

Pamiętajmy, że w przypadku średniej kroczącej nie mamy wyników dla kilku pierwszych wartości.

In [ ]:
ts_log_moving_avg_diff = ts_log - moving_avg
print(ts_log_moving_avg_diff.head(12))
ts_log_moving_avg_diff.dropna(inplace=True)
print(ts_log_moving_avg_diff.head(12))

# Zadanie 2
Sprawdź czy szereg czasowy jest już stacjonarny.

## Średnia ważona 

Jednakże wadą w tym szczególnym podejściu jest to, że okres czasu musi być ściśle określony. W tym przypadku możemy przyjąć średnie roczne, ale w skomplikowanych sytuacjach, takich jak prognozowanie cen akcji, trudno jest dobrać tą stałą. 

Przyjmujemy zatem "ważoną średnią ruchliwą", gdzie bliższe wartości mają wyższe wagi. Istnieje wiele technik przypisywania wag. My użyjemy średniej harmonicznej.

https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#exponentially-weighted-windows

# Zadanie 3
Wykonaj analogiczna zadanie przy użyciu średniej ważonej.

In [ ]:
expwighted_avg = ts_log.ewm(halflife=12).mean()

plt.figure(figsize=(12,6))
plt.plot(ts_log)
plt.plot(expwighted_avg, color='red')
plt.show()

# Zadanie 4
Sprawdź czy szereg czasowy jest już stacjonarny.